# Federal Reserve Data

The Fed has no API for their own data. The two most important data sets are: 
- Monthly Industrial Production and Capacity utilization.
- Monthly Consumer Credit Survey
- Quarterly Households Balance Sheet

They provide direct links to download data in .csv format.

In [25]:
import os
import pandas as pd
import requests
import csv
from io import StringIO

# path for the folder "project"
path = "C:\\Users\\pedro\\OneDrive\\NYU\\CSS\\II. Data Skills\\project"
os.chdir(path)

## Industrial Production

Importing Industrial Production data:

In [31]:
# Define the remote URL
url = "https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17&series=5d88c03b0036f0334d78f6bafefc5101&lastobs=&from=&to=&filetype=csv&label=include&layout=seriesrow&type=package"
# Send HTTP GET request via requests
data = requests.get(url)
# Convert to iterator by splitting on \n chars
lines = data.text.splitlines()

In [36]:
ip = (pd.concat([pd.read_csv(StringIO(lines[i]),header=None) for i in range(len(lines))]))
ip.columns = ip.reset_index(drop=True).loc[0,:]
ip = ip.iloc[1:,:].reset_index(drop=True)

In [33]:
ip.head()

,Descriptions:,Unit:,Multiplier:,Currency:,Unique Identifier:,Series Name:,1919-01,1919-02,1919-03,1919-04,...,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
0,Total index; s.a. IP,Index:_2017_100,1,NaN,G17/IP_MARKET_GROUPS/IP.B50001.S,IP.B50001.S,4.8773,4.6617,4.527,4.6078,...,102.146,102.8987,103.5717,104.2709,104.1646,104.0772,104.7577,104.6544,104.805,104.6894
1,Final products and nonindustrial supplies; s.a...,Index:_2017_100,1,NaN,G17/IP_MARKET_GROUPS/IP.B50030.S,IP.B50030.S,NaN,NaN,NaN,NaN,...,102.3739,103.3774,103.7037,104.7216,104.231,103.5966,104.1415,104.328,104.4301,104.6421
2,Consumer goods; s.a. IP,Index:_2017_100,1,NaN,G17/IP_MARKET_GROUPS/IP.B51000.S,IP.B51000.S,NaN,NaN,NaN,NaN,...,104.2414,104.4799,104.6884,106.0705,105.2305,104.4278,104.8702,104.9043,104.841,104.9898
3,Durable consumer goods; s.a. IP,Index:_2017_100,1,NaN,G17/IP_MARKET_GROUPS/IP.B51100.S,IP.B51100.S,NaN,NaN,NaN,NaN,...,112.3519,110.3115,113.5347,116.0243,113.2849,111.3795,113.8072,113.5913,112.9378,115.1849
4,Automotive products; s.a. IP,Index:_2017_100,1,NaN,G17/IP_MARKET_GROUPS/IP.B51110.S,IP.B51110.S,NaN,NaN,NaN,NaN,...,112.8143,108.2309,116.3315,119.724,118.9091,116.492,120.9201,119.4904,117.9411,121.5232


In [37]:
ip_dict = ip.iloc[:,[0,5]]
ip_dict.columns = ["item_name","series_id"]
ip_dict.head()

,item_name,series_id
0,Total index; s.a. IP,IP.B50001.S
1,Final products and nonindustrial supplies; s.a...,IP.B50030.S
2,Consumer goods; s.a. IP,IP.B51000.S
3,Durable consumer goods; s.a. IP,IP.B51100.S
4,Automotive products; s.a. IP,IP.B51110.S


In [42]:
ip = ip.iloc[:,5:].rename(columns={ip.columns[5]:"series_id"})

In [46]:
ip = ip.melt(id_vars="series_id", var_name="date").dropna()
ip["date"] = pd.to_datetime(ip["date"] + "-01")
ip.head()

,series_id,date,value
0,IP.B50001.S,1919-01-01,4.8773
33,IP.B00004.S,1919-01-01,4.8912
57,IP.G21.S,1919-01-01,23.5503
61,IP.B50001.S,1919-02-01,4.6617
94,IP.B00004.S,1919-02-01,4.7887


In [48]:
ip = pd.merge(ip,ip_dict, how="left", on = "series_id").reset_index(drop=True)
ip.info()
ip.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46654 entries, 0 to 46653
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   series_id  46654 non-null  object        
 1   date       46654 non-null  datetime64[ns]
 2   value      46654 non-null  object        
 3   item_name  46654 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.4+ MB


,series_id,date,value,item_name
0,IP.B50001.S,1919-01-01,4.8773,Total index; s.a. IP
1,IP.B00004.S,1919-01-01,4.8912,Manufacturing (SIC); s.a. IP
2,IP.G21.S,1919-01-01,23.5503,Mining (NAICS = 21); s.a. IP
3,IP.B50001.S,1919-02-01,4.6617,Total index; s.a. IP
4,IP.B00004.S,1919-02-01,4.7887,Manufacturing (SIC); s.a. IP


In [49]:
ip["value"] = pd.to_numeric(ip["value"])
ip["series_id"] = ip["series_id"].replace(".","")
ip.head()

,series_id,date,value,item_name
0,IP.B50001.S,1919-01-01,4.8773,Total index; s.a. IP
1,IP.B00004.S,1919-01-01,4.8912,Manufacturing (SIC); s.a. IP
2,IP.G21.S,1919-01-01,23.5503,Mining (NAICS = 21); s.a. IP
3,IP.B50001.S,1919-02-01,4.6617,Total index; s.a. IP
4,IP.B00004.S,1919-02-01,4.7887,Manufacturing (SIC); s.a. IP


Saving data

In [50]:
ip.to_parquet("data\\activity\\data_fed_industrial_production.parquet")